In [1]:
import pandas as pd
import ast  #mporta el módulo ast, que proporciona funciones para evaluar expresiones literales de Python.

In [2]:
ruta=r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos crudos\australian_users_items.json'
data=[]

with open(ruta, 'r', encoding='utf-8') as file:
    for linea in file:
        try:
            # Convertir cada línea del archivo JSON en un diccionario utilizando ast.literal_eval
            json_datos=ast.literal_eval(linea)
             # Agregar el diccionario a la lista de datos
            data.append(json_datos)
        except ValueError:
            # Manejar errores de formato en el JSON
            print(f"Error en la linea: {linea}")
            continue

# Crear un DataFrame de pandas a partir de la lista de datos
df_items=pd.DataFrame(data)

In [3]:
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [4]:
print(df_items.shape)
print(df_items.isnull().sum())

(88310, 5)
user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64


In [5]:
# Eliminar las columnas 'steam_id' y 'user_url' del DataFrame df_items

df_items.drop(columns=['steam_id','user_url'], inplace=True)

In [6]:
# Explotar la columna 'items' del DataFrame df_items
df_explode = df_items.explode('items').reset_index(drop=True)
# Normalizar los datos en la columna 'items' del DataFrame df_explode
items = pd.json_normalize(df_explode['items']).reset_index(drop=True)
# Concatenar los DataFrames df_explode y items a lo largo del eje de las columnas
df_items_full = pd.concat([df_explode.drop('items', axis=1), items], axis=1)

In [7]:
print(df_items_full.shape)
df_items_full.head()

(5170015, 6)


,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,50,Half-Life: Opposing Force,0.0,0.0


In [8]:
# Eliminar las columnas 'item_id' y 'item_name' del DataFrame df_items_full

df_items_full.drop(columns=['item_id','item_name'], inplace=True)

In [9]:
df_items_full.head()

,user_id,items_count,playtime_forever,playtime_2weeks
0,76561197970982479,277,6.0,0.0
1,76561197970982479,277,0.0,0.0
2,76561197970982479,277,7.0,0.0
3,76561197970982479,277,0.0,0.0
4,76561197970982479,277,0.0,0.0


In [10]:
# Eliminar las filas duplicadas del DataFrame df_items_full y crear un nuevo DataFrame df_items_clear

df_items_clear=df_items_full.drop_duplicates(keep='first')

In [11]:
# Guardar el DataFrame df_items_clear en formato Parquet

df_items_clear.to_parquet(r'C:\Users\Moises\Desktop\Projecto invidual 1\archivos limpios\items.parquet')